In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

def faustmann(revenue, age, cost, interest):
    if age:
        revenue = (revenue - (cost * interest ** age)) / ((interest ** age) - 1)
        return revenue
    else:
        return 0

 
    
#carbon_price = 3
#ll_timber = 0.72
#ll_pulp = 0.27
#hw_timber = 23.25/35.31467
#hw_pulp = 9.78/35.31467
    
ll_800 = pd.read_csv("/mnt/5A5136DD7377F658/rawdata/800LL.csv", delimiter = ";")
sg_800 = pd.read_csv("/mnt/5A5136DD7377F658/rawdata/800SG.csv", delimiter = ";")
wo_800 = pd.read_csv("/mnt/5A5136DD7377F658/rawdata/800WO.csv", delimiter = ";")
ll200_sg600 = pd.read_csv("/mnt/5A5136DD7377F658/rawdata/200LL_600SG.csv", delimiter = ";")
ll400_sg400 = pd.read_csv("/mnt/5A5136DD7377F658/rawdata/400LL_400SG.csv", delimiter = ";")
ll600_sg200 = pd.read_csv("/mnt/5A5136DD7377F658/rawdata/600LL_200SG.csv", delimiter = ";")
ll600_wo200 = pd.read_csv("/mnt/5A5136DD7377F658/rawdata/600LL_200WO.csv", delimiter = ";")
ll400_wo400 = pd.read_csv("/mnt/5A5136DD7377F658/rawdata/400LL_400WO.csv", delimiter = ";")
ll200_wo600 = pd.read_csv("/mnt/5A5136DD7377F658/rawdata/200LL_600WO.csv", delimiter = ";")

stands = [ll_800, sg_800, wo_800, ll200_sg600, ll400_sg400, ll600_sg200, ll600_wo200, ll400_wo400, ll200_wo600] 
stands_sg = [ll_800, ll600_sg200, ll400_sg400, ll200_sg600, sg_800]
stands_wo = [ll_800, ll600_wo200, ll400_wo400, ll200_wo600, wo_800]
  
def lev(graphtitle, ll_timber, ll_pulp, hw_timber, hw_pulp, carbon_price, cost, interest, *args):  
    for stand in args:
        stand_list = list(stand)
        stand.insert(loc=1, column="age", value=[i for i in range(0, 200, 5)])
        if len(stand_list) > 7:
            stand["ll_pulp"] = stand.iloc[:,3] - stand.iloc[:,6]
            stand["hw_pulp"] = stand.iloc[:,4] - stand.iloc[:,7]
            stand["revenue"] = stand.iloc[:,6] * ll_timber + stand.iloc[:,6] * hw_timber + stand.iloc[:,8] * ll_pulp + stand.iloc[:,9] * hw_pulp
            
        else:
            stand["pulp"] = stand.iloc[:,3] - stand.iloc[:,5]
            if stand.iloc[3,2] == 0:
                stand["revenue"] = stand.iloc[:,5] * hw_timber + stand.iloc[:,6] * hw_pulp
            else:
                stand["revenue"] = stand.iloc[:,5] * ll_timber + stand.iloc[:,6] * ll_pulp
                
        stand["faustmann"] = stand[["revenue", "age"]].apply(lambda x: faustmann(x["revenue"], x["age"], cost, interest), axis=1)
            
        stand["carbon_added"] = stand["Total_Stand_Carbon"].sub(stand["Total_Stand_Carbon"].shift(fill_value=0))
        stand["carbon_return"] = stand["carbon_added"] * carbon_price
        
       # plt.plot(stand["age"], stand["faustmann"])
        #plt.xlabel("age")
       # plt.ylabel("lev $/acre")
       # plt.title(graphtitle)
    #ax.plot(data = stand, x = "age", y = "faustmann", legend = "full")
    #fig, ax = sns.lineplot(data = stand, x = "age", y = "faustmann", legend = "full")
    #ax.ylabel('revenue in $/acre')
    #plt.title("revenue of stands by faustmann")
    #ax.legend()

lev("lev of all stands 800/acre", 0.72, 0.27, 23.25/35.31467, 0.1, 3, 262, 1.03, *stands)
#print(ll_800.head)


interest = 1.03
test = pd.DataFrame()
test["age"] = ll_800["age"]
test["income"] = ll_800["carbon_return"]

#age = test['age'].to_numpy()
#arr = interest ** (age[:, None] - age )
#arr = np.where(np.triu(np.ones(arr.shape, dtype=bool)), 0, arr)
#np.fill_diagonal(arr, 1)
#arr * test['income'].to_numpy()
#test['new'] = (arr * test['income'].to_numpy()).sum(axis=1)
#print(test.head)

test['sumproduct'] = (test['income'].expanding()
 .apply(lambda x: sum(test.loc[:x.index[-1], 'age'] * interest**(x.iloc[-1]-x)))
)
print(test.head)

<bound method NDFrame.head of     age     income   sumproduct
0     0  46.572121     0.000000
1     5  11.773018     5.000000
2    10  68.738914    36.931095
3    15  79.908989    66.378917
4    20  90.491318   110.756533
5    25  84.197227   116.954015
6    30  71.714012   110.865877
7    35  56.568191   105.854525
8    40  44.577942   114.265635
9    45  35.796387   133.142394
10   50  25.507965   148.229420
11   55  17.194977   170.936106
12   60  13.028229   211.127790
13   65   9.117599   253.080500
14   70   5.567001   297.865648
15   75   3.197525   352.717292
16   80   4.402817   445.510074
17   85   4.753647   535.154101
18   90   3.016571   598.369666
19   95   2.632965   686.623112
20  100   2.621246   786.385315
21  105   1.484665   865.404788
22  110   1.812927   983.842719
23  115   1.278808  1083.431869
24  120  -0.419952  1150.372436
25  125   0.784790  1317.076196
26  130   1.246078  1465.157696
27  135   0.144836  1553.232647
28  140   0.473099  1708.377086
29  145  -

In [25]:
def wood_volume(graphtitle, age, *args):
    volume_true = []
    volume_arithmetic = []
    for stand in args:
        volume_true.append(stand.loc[stand["age"] == age]["Total_MCuFt"].item())
        
    volume_arithmetic.append(volume_true[0])
    volume_arithmetic.append(volume_true[0] * 0.75 + volume_true[-1] * 0.25)
    volume_arithmetic.append(volume_true[0] * 0.5 + volume_true[-1] * 0.5)   
    volume_arithmetic.append(volume_true[0] * 0.25 + volume_true[-1] * 0.75)
    volume_arithmetic.append(volume_true[-1])
    
    stand_df = pd.DataFrame()
    stand_df["mix type"] = ["ll_100", "ll_75", "ll_50", "ll_25", "ll_0"]
    stand_df["true volume"] = volume_true
    stand_df["arithmetic"] = volume_arithmetic
    vol_comp = stand_df.plot(x="mix type", y="true volume", legend=False, marker = "o")
    plt.ylabel("wood volume in CuFt/acre")
    vol_comp2 = vol_comp.twinx()
    stand_df.plot(x="mix type", y="arithmetic", ax=vol_comp2, legend=False, marker = "o", color="r")
    plt.title(graphtitle + " at age " + str(age))
    plt.ylabel("wood volume in CuFt/acre")
    
#wood_volume("wood volume wo", 50, *stands_wo)

In [26]:
def carbon_volume(graphtitle, age, *args):
    volume_true = []
    volume_arithmetic = []
    for stand in args:
        volume_true.append(stand.loc[stand["age"] == age]["Total_Stand_Carbon"].item())
        
    volume_arithmetic.append(volume_true[0])
    volume_arithmetic.append(volume_true[0] * 0.75 + volume_true[-1] * 0.25)
    volume_arithmetic.append(volume_true[0] * 0.5 + volume_true[-1] * 0.5)   
    volume_arithmetic.append(volume_true[0] * 0.25 + volume_true[-1] * 0.75)
    volume_arithmetic.append(volume_true[-1])
    
    stand_df = pd.DataFrame()
    stand_df["mix type"] = ["ll_100", "ll_75", "ll_50", "ll_25", "ll_0"]
    stand_df["true volume"] = volume_true
    stand_df["arithmetic"] = volume_arithmetic
    vol_comp = stand_df.plot(x="mix type", y="true volume", legend=False, marker = "o")
    plt.ylabel("carbon sequestration in metric ton / hectare")
    vol_comp2 = vol_comp.twinx()
    stand_df.plot(x="mix type", y="arithmetic", ax=vol_comp2, legend=False, marker = "o", color="r")
    plt.title(graphtitle + " at age " + str(age))
    plt.ylabel("carbon sequestration in metric ton / hectare")

#carbon_volume("carbon sequestration in sg stands", 50, *stands_sg)

In [26]:
df = pd.DataFrame({"age" : [0, 5, 10, 15, 20], "income": [5, 13, 23, 18, 12]})


print(df)

   age  income  sumproduct
0    0       5           5
1    5      13          38
2   10      23         138
3   15      18         338
4   20      12         627
